Install libraries

In [ ]:
!pip install langchain langchain-huggingface chromadb transformers torch

2. Import Libraries

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import os, getpass


3. Authenticate Hugging Face

In [ ]:
# Prompt for your Hugging Face API key if not already set
if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your Hugging Face API key: ")


4. Load and Split CTSE Lecture Notes

In [ ]:
# Load your lecture notes PDF file
loader = PyPDFLoader("../CTSE_Lecture_Notes.pdf")  # Replace with your file name
documents = loader.load()

# Split into chunks (important for context-aware retrieval)
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = splitter.split_documents(documents)


5. Create Embeddings & Vector Store

In [ ]:
persist_directory = "./chroma_langchain_db"

# Use Sentence Transformer to convert text into vectors
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

if os.path.exists(persist_directory):
    # If already exists, load the existing DB
    vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

else:
    # Otherwise, create and save
    vector_store = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    vector_store.persist()


6. Set Up Retriever

In [ ]:
# Create a retriever that returns top 4 relevant chunks
retriever = vector_store.as_retriever(search_kwargs={"k": 8})


7. Initialize Flan-T5 Base LLM

In [ ]:
# Load Hugging Face's flan-t5-base model for QA generation
model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Create a text2text generation pipeline
flan_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
llm = HuggingFacePipeline(pipeline=flan_pipeline)


8. Build Retrieval QA Chain

In [ ]:
chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the provided context to answer the user's question. If you don't know the answer based on the context, say 'I don't know.'"),
    ("human", "Context:\n{context}\n\nQuestion: {question}"),
])

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
# Build Conversational Retrieval QA Chain correctly
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": chat_template}
)

9. Define Chatbot Function

In [ ]:
# A chatbot function
def chatbot(query: str) -> str:
    response = qa_chain.run({"question": query})
    return response

10. Batch Querying – Test Multiple Questions

In [ ]:
# Example questions
questions = [
    "What is software engineering?",
    "Explain the Agile methodology.",
    "What are the current trends in AI and ML?",
    "Define DevOps in software development.",
    "How does LangChain help in building chatbots?"
]

# Process all questions at once
for q in questions:
    print(f"\n🟡 Question: {q}")
    print(f"🟢 Answer: {chatbot(q)}")